In [1]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [2]:
def product_experiment_configs(params_record):
    # Generate all possible combinations of parameters
    keys = params_record.keys()
    values = params_record.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [3]:
common_params = {
    "params.dataset.path": ["bdsaglam/musique"],
    "params.dataset.split": ["validation"],
    "params.qa.model": ["llama-3-70b-tgi"],
    "params.qa.temperature": [0.1, 0.5],
    "params.qa.user_prompt_template": ["cq.txt"],
    "params.qa.n_shot": [0, 3, 6, 9],
    "params.run": [1, 2, 3],
}
params_records = [
    {
        "params.qa.system_prompt": [
            "direct/format-few.txt",
        ],
        "params.qa.few_shot_examples": ["auto/direct.json"],
    },
    {
        "params.qa.system_prompt": [
            "cot/format-thought.txt",
        ],
        "params.qa.few_shot_examples": ["auto/cot.json"],
    },
    {
        "params.qa.system_prompt": [
            "ccot/format-thought.txt",
        ],
        "params.qa.few_shot_examples": ["auto/ccot.json"],
    },
    {
        "params.qa.system_prompt": [
            "cte/format-triplets-few.txt",
            "cte/excellent-format-triplets-few.txt",
        ],
        "params.qa.few_shot_examples": ["auto/cte.json"],
    },
]

exp_configs = [
    exp_config
    for params_record in params_records
    for exp_config in product_experiment_configs({**common_params, **params_record})
]
print(f"{len(exp_configs)} experiment configurations")

120 experiment configurations


In [4]:
target_params = exp_configs[0].keys()
target_params

dict_keys(['params.dataset.path', 'params.dataset.split', 'params.qa.model', 'params.qa.temperature', 'params.qa.user_prompt_template', 'params.qa.n_shot', 'params.run', 'params.qa.system_prompt', 'params.qa.few_shot_examples'])

In [5]:
results_path = Path("results.jsonl")
if results_path.exists():
    with open(results_path) as f:
        experiments = [json.loads(line) for line in f]
else:
    experiments = []

print(f"{len(experiments)} experiments")
next(iter(experiments), None)

78 experiments


{'id': '0df05c2a7fc48f631142b9970aa9200383af59e4',
 'name': 'beamy-weka',
 'params.dataset.path': 'bdsaglam/musique',
 'params.dataset.name': 'answerable',
 'params.dataset.split': 'validation',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.temperature': 0.1,
 'params.qa.system_prompt': 'cot/format-thought.txt',
 'params.qa.user_prompt_template': 'cq.txt',
 'params.qa.few_shot_examples': 'auto/cot.json',
 'params.qa.n_shot': 6,
 'params.run': 2,
 'metrics.exact_match': 0.0024824162,
 'metrics.f1': 0.0049808358,
 'metrics.2hops.exact_match': 0.0015974441,
 'metrics.2hops.f1': 0.0052542833,
 'metrics.3hops.exact_match': 0.0039473684,
 'metrics.3hops.f1': 0.0055764411,
 'metrics.4hops.exact_match': 0.0024691358,
 'metrics.4hops.f1': 0.0030178326,
 'metrics.gen_token_count.all.count': 2417.0,
 'metrics.gen_token_count.all.mean': 37.7939594539,
 'metrics.gen_token_count.all.std': 304.1622038838,
 'metrics.gen_token_count.all.min': 0.0,
 'metrics.gen_token_count.all.25%': 1.0,
 'metrics

In [6]:
df = pd.DataFrame(experiments)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

78 experiments after preprocessing


In [7]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

In [8]:
target_params = list({**common_params, **params_records[0]}.keys())
target_params.remove("params.run")
target_params

['params.dataset.path',
 'params.dataset.split',
 'params.qa.model',
 'params.qa.temperature',
 'params.qa.user_prompt_template',
 'params.qa.n_shot',
 'params.qa.system_prompt',
 'params.qa.few_shot_examples']

In [9]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")
next(iter(missing_configs), None)


42 missing configurations


{'params.dataset.path': 'bdsaglam/musique',
 'params.dataset.split': 'validation',
 'params.qa.few_shot_examples': 'auto/direct.json',
 'params.qa.model': 'llama-3-70b-tgi',
 'params.qa.n_shot': 3,
 'params.qa.system_prompt': 'direct/format-few.txt',
 'params.qa.temperature': 0.1,
 'params.qa.user_prompt_template': 'cq.txt',
 'params.run': 3}

In [10]:
def make_command(exp_config):
    run = exp_config["params.run"]
    lines = ["dvc exp run --queue", f"-S run={run}"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [11]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")